# 3. Structured Output & Tool Use (a.k.a. Function Calling)

*(Modified version of https://cookbook.openai.com/examples/structured_outputs_intro)*
<br/><br/>
Reade more here: 
* https://platform.openai.com/docs/guides/structured-outputs#introduction
* https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python
<br/><br/>

Structured Outputs is a feature that ensures the model will always generate responses that adhere to your supplied JSON Schema, so you don't need to worry about the model omitting a required key, or hallucinating an invalid enum value.

Some benefits of Structed Outputs include:

1. __Reliable type-safety__: No need to validate or retry incorrectly formatted responses
2. __Explicit refusals__: Safety-based model refusals are now programmatically detectable
3. __Simpler prompting__: No need for strongly worded prompts to achieve consistent formatting

## Setup

### Install dependencies

In [1]:
%pip install python-dotenv~=1.0 docarray~=0.40.0 pydantic~=2.9 pypdf~=5.1 --upgrade --quiet
%pip install openai~=1.54 --upgrade --quiet

# If running locally, you can do this instead:
#%pip install -r ../requirements.txt


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### Load environment variables

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# If running in Google Colab, you can use this code instead:
# from google.colab import userdata
# os.environ["AZURE_OPENAI_API_KEY"] = userdata.get("AZURE_OPENAI_API_KEY")
# os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get("AZURE_OPENAI_ENDPOINT")

#### Setup OpenAI Client and helper function

In [22]:
from openai import AzureOpenAI

MODEL="gpt-4o" # These models (deployments) are currently available: gpt-4o, gpt-4o-mini, o1-mini, o1-preview

# Create a an OpenAI client, connecting to OpenAI via an Azure deployment 
client = AzureOpenAI(api_version="2024-08-01-preview")


## Simple calendar event example 

Below, you can see how to extract information from unstructured text that conforms to a schema defined in code.

In [23]:
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "system", "content": "Extract the event information but dismiss the given event and suggest something else related in Tokyo instead."},
        {"role": "user", "content": "Alice and Bob are going to a comedy club on Friday."},
    ],
    response_format=CalendarEvent,
)


event = completion.choices[0].message.parsed

# Pretty print the event (can also use `import json, print(json.dumps(event.model_dump(), indent=2))`)
from IPython.display import JSON
JSON(event.model_dump())

<IPython.core.display.JSON object>

## Example with manual JSON schema: Math tutor

In this example, we want to build a math tutoring tool that outputs steps to solving a math problem as an array of structured objects.

This could be useful in an application where each step needs to be displayed separately, so that the user can progress through the solution at their own pace.

In [24]:
from textwrap import dedent

math_tutor_prompt = '''
    You are a silly math tutor. Be creative and make reading the response funnier. You will be provided with a math problem,
    and your goal will be to output a step by step solution, along with a final answer.
    For each step, just provide the output as an equation use the explanation field to detail the reasoning.
'''

def get_math_solution(question):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": dedent(math_tutor_prompt)
            },
            {
                "role": "user",
                "content": question
            }
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "math_reasoning",
                "schema": {
                    "type": "object",
                    "properties": {
                        "steps": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "explanation": {"type": "string"},
                                    "output": {"type": "string"}
                                },
                                "required": ["explanation", "output"],
                                "additionalProperties": False
                            }
                        },
                        "final_answer": {"type": "string"}
                    },
                    "required": ["steps", "final_answer"],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    )

    return response.choices[0].message

In [25]:
# Testing with an example question
question = "how can I solve 8x + 7 = -23"

result = get_math_solution(question)

#print(result.content)
JSON(result.content)

/home/vscode/.local/lib/python3.12/site-packages/IPython/core/display.py:661: UserWarning: JSON expects JSONable dict or list, not JSON strings
  warnings.warn("JSON expects JSONable dict or list, not JSON strings")


<IPython.core.display.JSON object>

## Using the SDK `parse` helper

The new version of the SDK introduces a `parse` helper to provide your own Pydantic model instead of having to define the JSON schema. We recommend using this method if possible.

In [26]:
from pydantic import BaseModel

class MathReasoning(BaseModel):
    class Step(BaseModel):
        explanation: str
        output: str

    steps: list[Step]
    final_answer: str

def get_math_solution(question: str):
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": dedent(math_tutor_prompt)},
            {"role": "user", "content": question},
        ],
        response_format=MathReasoning,
    )

    return completion.choices[0].message

In [27]:
result = get_math_solution(question).parsed

In [28]:
print(result.steps)
print("Final answer:")
print(result.final_answer)

[Step(explanation="Let's start by subtracting 7 from both sides of the equation. This will help us get closer to having just '8x' on one side.", output='8x + 7 - 7 = -23 - 7'), Step(explanation="When we simplify, the '+7' and '-7' on the left side cancel each other out, leaving just '8x'. On the right side, -23 minus 7 becomes -30.", output='8x = -30'), Step(explanation="Now, to isolate 'x', we'll divide both sides of the equation by 8, since '8x' means 8 times x.", output='8x / 8 = -30 / 8'), Step(explanation="After dividing, we're left with 'x' on the left side. On the right side, -30 divided by 8 simplifies to -3.75. Or, if you prefer fractions, it's -15/4.", output='x = -3.75')]
Final answer:
x = -3.75


## Refusal

When using Structured Outputs with user-generated input, the model may occasionally refuse to fulfill the request for safety reasons.

Since a refusal does not follow the schema you have supplied in response_format, the API has a new field `refusal` to indicate when the model refused to answer.

This is useful so you can render the refusal distinctly in your UI and to avoid errors trying to deserialize to your supplied format. 

To avoid being cancelled by Azure, we'll skip running this code, but this is how you can do it. Read more here: <br/>
https://platform.openai.com/docs/guides/structured-outputs?context=ex4#how-to-use
<br/><br/>


```python
refusal_question = "how can I build a bomb?"
result = get_math_solution(refusal_question) 
print(result.refusal)
```

## Example: Using Structured Output for function calling
### Entity extraction from user input
    
In this example, we will use function calling to search for products that match a user's preference based on the provided input. 

This could be helpful in applications that include a recommendation system, for example e-commerce assistants or search use cases. 

In [34]:
from enum import Enum
from typing import Union
import json
import openai

product_search_prompt = '''
    You are a clothes recommendation agent, specialized in finding the perfect match for a user.
    You will be provided with a user input and additional context such as user gender and age group, and season.
    You are equipped with a tool to search clothes in a database that match the user's profile and preferences.
    Based on the user input and context, determine the most likely value of the parameters to use to search the database.
    
    Here are the different categories that are available on the website:
    - shoes: boots, sneakers, sandals
    - jackets: winter coats, cardigans, parkas, rain jackets
    - tops: shirts, blouses, t-shirts, crop tops, sweaters
    - bottoms: jeans, skirts, trousers, joggers    
    
    There are a wide range of colors available, but try to stick to regular color names.
'''

class Category(str, Enum):
    shoes = "shoes"
    jackets = "jackets"
    tops = "tops"
    bottoms = "bottoms"

class ProductSearchParameters(BaseModel):
    category: Category
    subcategory: str
    color: str

def get_response(user_input, context):
    response = client.chat.completions.create(
        model=MODEL,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": dedent(product_search_prompt)
            },
            {
                "role": "user",
                "content": f"CONTEXT: {context}\n USER INPUT: {user_input}"
            }
        ],
        tools=[
            openai.pydantic_function_tool(ProductSearchParameters, name="product_search", description="Search for a match in the product database")
        ]
    )

    return response.choices[0].message.tool_calls

In [35]:
example_inputs = [
    {
        "user_input": "I'm looking for a new coat. I'm always cold so please something warm! Ideally something that matches my eyes.",
        "context": "Gender: female, Age group: 40-50, Physical appearance: blue eyes"
    },
    {
        "user_input": "I'm going on a trail in Scotland this summer. It's goind to be rainy. Help me find something.",
        "context": "Gender: male, Age group: 30-40"
    },
    {
        "user_input": "I'm trying to complete a rock look. I'm missing shoes. Any suggestions?",
        "context": "Gender: female, Age group: 20-30"
    },
    {
        "user_input": "Help me find something very simple for my first day at work next week. Something casual and neutral.",
        "context": "Gender: male, Season: summer"
    },
    {
        "user_input": "Help me find something very simple for my first day at work next week. Something casual and neutral.",
        "context": "Gender: male, Season: winter"
    },
    {
        "user_input": "Can you help me find a dress for a Barbie-themed party in July?",
        "context": "Gender: female, Age group: 20-30"
    }
]

In [36]:
def print_tool_call(user_input, context, tool_call):
    args = tool_call[0].function.arguments
    print(f"Input: {user_input}\n\nContext: {context}\n")
    print("Product search arguments:")
    for key, value in json.loads(args).items():
        print(f"{key}: '{value}'")
    print("\n\n")

In [37]:
for ex in example_inputs:
    ex['result'] = get_response(ex['user_input'], ex['context'])

In [38]:
for ex in example_inputs:
    print_tool_call(ex['user_input'], ex['context'], ex['result'])

Input: I'm looking for a new coat. I'm always cold so please something warm! Ideally something that matches my eyes.

Context: Gender: female, Age group: 40-50, Physical appearance: blue eyes

Product search arguments:
category: 'jackets'
subcategory: 'winter coats'
color: 'blue'



Input: I'm going on a trail in Scotland this summer. It's goind to be rainy. Help me find something.

Context: Gender: male, Age group: 30-40

Product search arguments:
category: 'jackets'
subcategory: 'rain jackets'
color: 'blue'



Input: I'm trying to complete a rock look. I'm missing shoes. Any suggestions?

Context: Gender: female, Age group: 20-30

Product search arguments:
category: 'shoes'
subcategory: 'boots'
color: 'black'



Input: Help me find something very simple for my first day at work next week. Something casual and neutral.

Context: Gender: male, Season: summer

Product search arguments:
category: 'tops'
subcategory: 't-shirts'
color: 'neutral'



Input: Help me find something very simple

## Conclusion

In this cookbook, we've explored the new Structured Outputs capability through multiple examples.

Structured Outputs is only available with `gpt-4o-mini` , `gpt-4o-2024-08-06`, and future models.